In [ ]:
import networkx as nx
import random
import matplotlib.pyplot as plt

from networkx.drawing.nx_agraph import graphviz_layout, to_agraph

from itertools import combinations

In [ ]:
node_size = 300
font_size = 13
width = 5

In [ ]:
init = 'initial'
final = 'final'
label = 'label'
in_encoding = 'in_encoding'
in_current = 'in_current'
children = 'children'

In [ ]:
def plot_encoding_tree(t, figsize=None, transition=None, title=None, plot_encoding=False):
    figsize = (12, 6) if not figsize else figsize
    plt.figure(figsize=figsize)
    plt.subplot(1, 2, 1)
    if title: 
        plt.title(title)
    plt.axis('off')
    pos = graphviz_layout(t, prog='dot')
#     pos = nx.spectral_layout(t)
    if plot_encoding:
        nx.draw_networkx_nodes(t, pos, nodelist=[u for u in t if t.nodes[u][in_encoding]], 
                           node_size=node_size*2, node_color='white',linewidths=width, edgecolors='m')
    else:
        # plot the complement
        nx.draw_networkx_nodes(t, pos, nodelist=[u for u in t if not t.nodes[u][in_current]], 
                           node_size=node_size*2, node_color='white',linewidths=width, edgecolors='c')
    nx.draw_networkx_nodes(t, pos, nodelist=[u for u in t if t.nodes[u][in_current]], 
                           node_size=node_size*1.2, node_color='white',linewidths=width, edgecolors='g')
    nx.draw_networkx_nodes(t, pos, nodelist=[u for u in t if t.nodes[u][label]==init], 
                           node_size=node_size, node_color='b', alpha=1)
    nx.draw_networkx_nodes(t, pos, nodelist=[u for u in t if t.nodes[u][label]==final], 
                           node_size=node_size, node_color='r', alpha=1)
    nx.draw_networkx_labels(t, pos, font_size=font_size)
    nx.draw_networkx_edges(t, pos, width=1.5)
    
    plt.subplot(1, 2, 2)
    plt.axis('off')
    nodes = [u for u in t if t.nodes[u][in_encoding] or t.nodes[u][in_current]]
    ct = nx.induced_subgraph(t, nodes)
    nx.draw_networkx_nodes(ct, pos, nodelist=[u for u in t if t.nodes[u][in_encoding]], 
                           node_size=node_size*2, node_color='white',linewidths=width, edgecolors='m')
    nx.draw_networkx_nodes(ct, pos, nodelist=[u for u in t if t.nodes[u][in_current]], 
                           node_size=node_size*1.2, node_color='white',linewidths=width, edgecolors='g')
    nx.draw_networkx_labels(ct, pos, font_size=font_size)
    nx.draw_networkx_edges(ct, pos, width=1.5)

In [ ]:
def color_the_tree(t, root, color):
    t.nodes[root][label] = init if color == 0 else final
    for u in t.successors(root):
        color_the_tree(t, u, 1-color)

In [ ]:
def generate_tree(n):
    t = nx.random_tree(n)
#     t = nx.path_graph(n)
    root = random.sample(t.nodes, 1)[0]
    root = min([u for u in t if t.degree(u) == 1])
    t = nx.dfs_tree(t, root)
    color_the_tree(t, root, random.choice([0, 1]))
    for u in t:
        t.nodes[u][in_current] = t.nodes[u][label] == init
        t.nodes[u][in_encoding] = t.nodes[u][label] != init
    return t, root

In [ ]:
def get_parent(t, u):
    pre = list(t.predecessors(u))
    return pre[0] if len(pre) > 0 else None

In [ ]:
def get_neighbors_of_status(t, root, status):
    parent = get_parent(t, root)
    neighbors_of_status = [parent] if parent is not None and t.nodes[parent][status] else []
    neighbors_of_status.extend([u for u in t.successors(root) if t.nodes[u][status]])
    return neighbors_of_status

In [ ]:
def longest_path(t):
    leaves = [u for u in t if t.out_degree(u) == 0]
    und_t = nx.Graph(t)
    lengths_paths = []
    for u, v in combinations(leaves, 2):
        path = nx.shortest_path(und_t, source=u, target=v)
        lengths_paths.append((len(path), path))
    lengths_paths.sort(reverse=True)
    return lengths_paths[0][1]

In [ ]:
def update_encoding_for_blue(t, blue, figsize):
    # try to add blue to encoding
    if t.nodes[blue][in_current]:
        return
    neighbors_in_encoding = get_neighbors_of_status(t, blue, in_encoding)
    n = len(neighbors_in_encoding) 
    if n >= 2:
        return
    drag_encoding = False
    if n == 1:
        target = neighbors_in_encoding[0]
        t.nodes[target][in_encoding] = False
        action = f"update encoding for blue: {target} to {blue}"
        drag_encoding = True
    else:
        action = f"update encoding for blue: add {blue}"
    t.nodes[blue][in_encoding] = True
    plot_encoding_tree(t, title=action, figsize=figsize)
#     if drag_encoding:
#         update_encoding_for_red(t, red, figsize)

In [ ]:
# def update_encoding_for_red(t, red, figsize):
#     if t.nodes[red][in_encoding]:
#         return
#     neighbors_in_encoding = get_neighbors_of_status(t, red, in_encoding)
#     n = len(neighbors_in_encoding) 
#     if n >= 2:
#         return
#     drag_encoding = False
#     if n == 1:
#         target = neighbors_in_encoding[0]
#         t.nodes[target][in_encoding] = False
#         action = f"update encoding for red: {target} to {red}"
#         drag_encoding = True
#     else:
#         action = f"update encoding for red: add {b}"
#     t.nodes[blue][in_encoding] = True
#     plot_encoding_tree(t, action, figsize)
# #     if drag_encoding:
# #         update_encoding_for_red(t, red, figsize)

In [ ]:
def process_blue(t, blue, figsize):
    # delete or push to parent
    if not t.nodes[blue][in_current]:
        return 
    parent = get_parent(t, blue)
    if parent is None or t.nodes[parent][in_current]:
        return
    parent_neighbors_in_current = get_neighbors_of_status(t, parent, in_current)
    n = len(parent_neighbors_in_current)
    if n == 1:
        t.nodes[parent][in_current] = True
        action = f"process blue: drag {blue} to {parent}"
    else:
        action = f"process blue: remove {blue}"
    t.nodes[blue][in_current] = False
    plot_encoding_tree(t, title=action, figsize=figsize)
#     update_encoding_for_blue(t, blue, figsize)

In [ ]:
def process_red(t, red, figsize):
    # drag or remove
    if t.nodes[red][in_current]:
        return
    neighbors_in_current = get_neighbors_of_status(t, red, in_current)
    n = len(neighbors_in_current)
    if n >= 2:
        return
    drag_blue = False
    if n == 1:
        target = neighbors_in_current[0]
        t.nodes[target][in_current] = False
        action = f"process red: drag {target} to {red}"
        drag_blue = True
    else:
        action = f'process red: add {red}'
    t.nodes[red][in_current] = True
    t.nodes[red][in_encoding] = False
    plot_encoding_tree(t, title=action, figsize=figsize)
#     if drag_blue:
#         update_encoding_for_blue(t, target, figsize)

In [ ]:
# do we need propagation?
def process(t, root, figsize):
    root_label = t.nodes[root][label]
#     if root_label == init:
#         process_blue(t, root, figsize)
#     if root_label == final:
#         process_red(t, root, figsize)
    for u in t.successors(root):
        process(t, u, figsize)
    if root_label == init:
        process_blue(t, root, figsize)
    if root_label == final:
        process_red(t, root, figsize)

In [ ]:
t, root = generate_tree(20)

In [ ]:
figsize = (12, 6)

In [ ]:
plot_encoding_tree(t, figsize)

In [ ]:
print(longest_path(t))

In [ ]:
ct = nx.DiGraph(t)
process(ct, root, figsize)

In [ ]:
### order is a very important problem